### Step 1: Setting Up libraries

#### Installing libraries

- earthengine-api (Google Earth Engine API) for accessing Google Earth Engine
- geemap (Google Earth Engine Python API) for visualizing and analyzing Earth Engine datasets interactively in a Jupyter notebook
- geetools (Community tools for Google Earth Engine Python API) for helping with tasks such as batch processing and exporting images from Earth Engine

In [1]:
# %pip install earthengine-api geetools geemap pycrs GDAL

#### Upgrading Earth Engine Python API

In [2]:
# %pip install earthengine-api geetools --upgrade

#### Importing libraries

In [3]:
import ee
import geemap
import geetools as gt
from ipywidgets import Layout

Map = geemap.Map(layout=Layout(height='600px'))
Map.add_basemap('HYBRID')

#### Initialization

In [4]:
# ee.Authenticate()

In [5]:
ee.Initialize()

In [6]:
SFCC = {'bands': ['B8', 'B4', 'B3'], 'max': 3000}
roi = geemap.shp_to_ee("./Data/sangamner_shp/sangamner.shp")
s2 = ee.ImageCollection('COPERNICUS/S2').filterDate('2023-01-01', '2023-02-02').filter(
    ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 5)).filterBounds(roi)
# Map.centerObject(roi, 10)

In [7]:
SCALE = 10                # export resolution: 30m/px
# image dimension = (2*EXPORT_TILE_RADIUS) + 1 = 255px
EXPORT_TILE_RADIUS = 127
LABEL = 'Class'           # name of the label band
BANDS = ['B2', 'B3', 'B4', 'B8']
# Number of label values, i.e. number of classes in the classification.
N_CLASSES = 4
# These names are used to specify properties in the export of
# training/testing data and to define the mapping between names and data
# when reading into TensorFlow datasets.
FEATURE_NAMES = BANDS + [LABEL]

FOLDER = "GEETFR"

In [8]:
s2

Name,Description
B1,Aerosols
B2,Blue
B3,Green
B4,Red
B5,Red Edge 1
B6,Red Edge 2
B7,Red Edge 3
B8,NIR
B8A,Red Edge 4
B9,Water vapor


In [9]:
s2_list = s2.toList(s2.size())
s2_list

In [10]:
fcc_img = ee.Image(s2_list.get(2)).clip(roi)
Map.addLayer(fcc_img, SFCC, 'FCC')

In [11]:
# display(Map)

In [12]:
image_index = [2,4,6,8]
image_collection = ee.ImageCollection([ee.Image(s2_list.get(i)).clip(roi) for i in image_index]).select(['B2', 'B3', 'B4', 'B8'])
image_collection

In [13]:
def stackCollection(imgCollection: ee.ImageCollection) -> ee.Image:
    first = ee.Image(imgCollection.first())
    appendBands = lambda img, previous: ee.Image(previous).addBands(img)
    return ee.Image(imgCollection.iterate(appendBands, first))


In [14]:
image_stack = stackCollection(image_collection)
image_stack

In [15]:
# Importing Collections
agri = geemap.geojson_to_ee("./Data/featureCollections/Agricultural_Land.geojson")
barren = geemap.geojson_to_ee("./Data/featureCollections/Barren.geojson")
buildup = geemap.geojson_to_ee("./Data/featureCollections/BuildUp.geojson")
water = geemap.geojson_to_ee("./Data/featureCollections/Water.geojson")

In [16]:
# Sampling the collections
# combinedFC = agri.merge(water).merge(buildup).merge(barren) # Merging all the collections
# combinedFC = combinedFC.randomColumn() # Adding a random column to the collection
# sample = fcc_img.sampleRegions(
#     collection=combinedFC,
#     properties=[LABEL],
# )
# # Partition the sample approximately 80-20.
# training = sample.filter(ee.Filter.lt('random', 0.8))
# testing = sample.filter(ee.Filter.gte('random', 0.2))

In [17]:
# training_task = ee.batch.Export.table.toDrive(
#     folder=FOLDER,
#     collection=training,
#     description='Training Data',
#     fileNamePrefix="Training_demo",
#     selectors= FEATURE_NAMES,
#     fileFormat='TFRecord'
# )
# validation_task = ee.batch.Export.table.toDrive(
#     folder=FOLDER,
#     collection=testing,
#     description='Validation Data',
#     fileNamePrefix="Validation_demo",
#     selectors= FEATURE_NAMES,
#     fileFormat='TFRecord'
# )

In [18]:
# training_task.start()
# validation_task.start()

In [19]:
# test = agri.first()
# feature = ee.Feature(test)
# h_w = 100 #hxw meters actually 1px extra on each dim
# buffer_out = feature.buffer(int(h_w/2))
# buffer_out = buffer_out.bounds()
# fthis= fcc_image.clip(buffer_out)
# Map.addLayer(fthis, SFCC,"idk")
# Map.centerObject(buffer_out)

In [43]:
h_w = 100 #hxw meters actually 1px extra on each dim
def getPatch(feature: ee.Feature, image: ee.Image) -> tuple[ee.Image, ee.Geometry]:
    buffer_out = feature.buffer(int(h_w/2))
    buffer_out = buffer_out.bounds()
    patch= image.clip(buffer_out)
    return patch, buffer_out

def buildImageCollection(featureCollection: ee.FeatureCollection, image: ee.Image) -> ee.ImageCollection:
    f = lambda feature, previous: ee.ImageCollection(previous).merge(getPatch(feature, image)[0])
    return ee.ImageCollection(featureCollection.iterate(f, ee.ImageCollection([])))

def mapImageGeometry(featureCollection: ee.FeatureCollection, image:ee.Image) -> list:
    imageGeoList = []
    fColSize = featureCollection.size().getInfo()
    fColList = featureCollection.toList(fColSize)
    for i in range(fColSize):
        feature = ee.Feature(fColList.get(i))
        patch, buffer_out = getPatch(feature, image)
        item = {'image': patch, 'bounds': buffer_out}
        imageGeoList.append(item)
    return imageGeoList

    

In [38]:
agriImages = buildImageCollection(agri, fcc_img)
waterImages = buildImageCollection(water, fcc_img)
buildUpImages = buildImageCollection(buildup, fcc_img)
barrenImages = buildImageCollection(barren, fcc_img)

In [44]:
fuckThis = mapImageGeometry(water, fcc_img)
nthImage = fuckThis[0]

In [45]:
img1 = waterImages.first()
Map.addLayer(agriImages, SFCC, "agriBounds")
Map.addLayer(waterImages, SFCC, "waterBounds")
Map.addLayer(buildUpImages, SFCC, "buildUpBounds")
Map.addLayer(barrenImages, SFCC, "barrenBounds")
Map.centerObject(nthImage['bounds'], 20)

In [41]:
display(Map)

Map(center=[19.583945296290103, 74.21280640159063], controls=(WidgetControl(options=['position', 'transparent_…

In [ ]:
# RESOLUTION = 10
# test = geemap.ee_to_geotiff(agriBounds, resolution=10, vis_params=SFCC, output="./Data/test/Agricultural_Land.tif")

In [ ]:
# # Exactly 100x100 kernel
# kernel = ee.Kernel.fixed(100, 100, [[1] * 100] * 100)

# neighborhoodImage = image_stack.neighborhoodToArray(kernel)
# samples = neighborhoodImage.sampleRegions(collection=agri)
# exp_task = ee.batch.Export.table.toDrive(
#     folder="GEE",
#  collection=samples,
#  fileFormat='TFRecord')

In [ ]:
# exp_task.start()

In [ ]:
# SCALE=10 #meters per pixel

In [ ]:
# agri_task.start()